In [1]:
from json import JSONDecodeError
import time
import requests
import datetime
import pandas as pd
import concurrent.futures

def get_data(url):
    start_time = time.time() * 1000  # record the start time in milliseconds
    response = requests.get(url+'date?full_format=true')
    end_time = time.time() * 1000  # record the end time in milliseconds
    response_code = response.status_code
    hour = datetime.datetime.now().strftime("%H:%M:%S")
    counter_response = requests.get(f"{url}counter")
    try:
        count = counter_response.json()

    except JSONDecodeError:
        count = "N/A"
    return {"response_code": response_code, "response_hour": hour, "count": count, "response_time": end_time - start_time}


In [2]:
def construct_dataset(url,quantity_of_requests = 100):
    data = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(get_data, url) for _ in range(quantity_of_requests)]
        for future in concurrent.futures.as_completed(futures):
            data.append(future.result())
    df = pd.DataFrame(data)
    return df


In [3]:
apis_urls = {"python_api": "http://0.0.0.0:8081/",
             "golang_api": "http://localhost:8080/"}

for api_name, url in apis_urls.items():
    df = construct_dataset(url,quantity_of_requests=10000)
    df.to_csv(f"{api_name}_data.csv")
    del df


In [4]:
df_python = pd.read_csv("python_api_data.csv")

df_python.head(3)

,Unnamed: 0.1,Unnamed: 0,response_code,reponse_hour,count,response_time
0,0,0,405,17:42:44,8,87.667236
1,1,1,405,17:42:44,7,77.796875
2,2,2,405,17:42:44,6,135.043701


In [5]:
df_golang = pd.read_csv("golang_api_data.csv")
df_golang.head(3)

,Unnamed: 0.1,Unnamed: 0,response_code,reponse_hour,count,response_time
0,0,0,405,17:45:06,5,179.286865
1,1,1,405,17:45:07,8,65.072754
2,2,2,405,17:45:06,1,158.510986
